<h1 style="text-align:center;">Лабораторная работа №5</h1>

<h2 style="text-align:center;">Вычисление значения определенного интеграла</h2>

<h3 style="text-align:center;">Тулупов Николай Дмитриевич</h3>
<h3 style="text-align:center;">Б01-204</h3>


Цель: решить выбранную задачу перечисленными методами: 
1. Метод трапеций + уточнение по правилу Рунге
2. Формула Симпсона

<img src="lab_5_task_1.png" alt="drawing" width="650"/>


<img src="lab_5_task_2.png" alt="drawing" width="650"/>

In [1127]:
%matplotlib inline
import numpy as np
from scipy.interpolate import interp1d

import matplotlib.pyplot as plt